## 把gt和pred给设置成不同的数值，overlap的设置成别的数值

In [46]:
import os
import numpy as np
import torchio as tio

In [58]:
def bbox_iou(box1, box2):
    """
    计算两个边界框的交并比(IoU)。
    边界框的格式为 [x1, y1, z1, x2, y2, z2]。
    """
    # 计算交集的坐标
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    z1_inter = max(box1[2], box2[2])
    x2_inter = min(box1[3], box2[3])
    y2_inter = min(box1[4], box2[4])
    z2_inter = min(box1[5], box2[5])

    # 计算交集的体积
    inter_volume = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter) * max(0, z2_inter - z1_inter)

    # 计算每个边界框的体积
    box1_volume = (box1[3] - box1[0]) * (box1[4] - box1[1]) * (box1[5] - box1[2])
    box2_volume = (box2[3] - box2[0]) * (box2[4] - box2[1]) * (box2[5] - box2[2])

    # 计算并集的体积
    union_volume = box1_volume + box2_volume - inter_volume

    # 计算 IoU
    iou = inter_volume / union_volume

    return iou

def find_common_boxes(pred1, pred2, iou_threshold):
    """ 找出 pred1 和 pred2 中共有的边界框 """
    common_boxes = []
    for box1 in pred1:
        for box2 in pred2:
            if bbox_iou(box1, box2) >= iou_threshold:
                common_boxes.append(box2)  # 或 box2，取决于你的需求
    return common_boxes


def get_fp(gt, pred, iou_confi): # 得到pred中多预测出来的，就是pred中有，gt中没有的
    result = []
    for box in pred:
        overlap = False
        for p_box in gt:
            IoU = bbox_iou(box, p_box)
            if IoU >= iou_confi:
                overlap = True
                break
        if not overlap:
            result.append(box)

    return result


def classify_gt_boxes(gt_boxes, pred1_boxes, pred2_boxes, iou_threshold):
    gt_in_both, gt_in_pred1, gt_in_pred2, gt_in_none = [], [], [], []
    removed_fp = []
    def has_overlap(box, boxes, iou_threshold):
        return any(bbox_iou(box, other_box) >= iou_threshold for other_box in boxes)
    # print(f'gt is {gt_boxes}')
    # print(f'len gt is {len(gt_boxes)}')
    for gt_box in gt_boxes:
        in_pred1 = has_overlap(gt_box, pred1_boxes, iou_threshold)
        in_pred2 = has_overlap(gt_box, pred2_boxes, iou_threshold)
        # print(f'in_preds is {in_pred1}')
        if in_pred1 and in_pred2:
            gt_in_both.append(gt_box)
        elif in_pred1:
            gt_in_pred1.append(gt_box)
        elif in_pred2:
            gt_in_pred2.append(gt_box)
        else:
            gt_in_none.append(gt_box)

    fp_1 = get_fp(gt_boxes, pred1_boxes, iou_threshold)
    fp_2 = get_fp(gt_boxes, pred2_boxes, iou_threshold)
    fp_always = find_common_boxes(fp_1, fp_2, iou_threshold)
    removed_fp = get_fp(fp_2, fp_1, iou_threshold)
    fp_add = get_fp(fp_1, fp_2, iou_threshold)
    if len(gt_boxes) != (len(gt_in_both) + len(gt_in_none) + len(gt_in_pred1) + len(gt_in_pred2)):
        print(f'sothing wrong with the gt classfication')
    return gt_in_none, gt_in_pred1, gt_in_pred2, fp_always, gt_in_both, removed_fp, fp_add





In [59]:
import torchio as tio
import torch
# 读取gt(设置成为1)
def get_gtboxes(folder_path, filename):
    result = []
    if filename.endswith('.txt'):
        with open(os.path.join(folder_path, filename), 'r') as f:
            for line in f:
                # print(line)
                data = line.strip().split()
                x1, y1, z1, x2, y2, z2 = map(float, data[1:7])
                result.append([x1, y1, z1, x2, y2, z2])
                shape = [int(data[7].split(',')[0][1:]), int(data[7].split(',')[1][:]), int(data[7].split(',')[2][:-1])]
    # print(shape)
    return result, shape



def create_volume_with_boxes(shape, boxes_list, value):
    """
    创建一个三维数组，其中给定的框内的元素为 1,其他为 0。

    :param shape: 三维数组的形状，形如 (x_dim, y_dim, z_dim)
    :param boxes: 框的列表，每个框的形式为 [x1, y1, z1, x2, y2, z2]
    :return: 三维数组
    """
    # 创建一个全零的三维数组
    volume = np.zeros(shape)
    for boxes, v in zip(boxes_list, value):
        # 遍历所有的框并在数组中将对应的区域设为 1
        for box in boxes:
            x1, y1, z1, x2, y2, z2 = map(int, box)
            volume[x1:x2, y1:y2, z1:z2] = v

    return volume



# 读取pred(设置成为2)
def get_predboxes(pred_file_name, filename, confidence):
    result = []
    if filename.endswith('.txt'):   
        txt_path = os.path.join(pred_file_name, filename)
        if os.path.exists(txt_path):
            with open(txt_path, 'r') as f:
                for line in f:
                    data = line.strip().split()
                    confi = float(data[1])
                    if confi >= confidence:
                        x1, y1, z1, x2, y2, z2 = map(float, data[2:8])
                        result.append([x1, y1, z1, x2, y2, z2])
                        shape = [int(data[8].split(',')[0][1:]), int(data[8].split(',')[1][:]), int(data[8].split(',')[2][:-1])]
        else:
            result.append([0., 0., 0., 0., 0., 0.])
            shape = [0, 0, 0]
            print(f'in the {filename} no pred bbox!')
    return result, shape


# overlap(设置成为3)(两者相加即可)
folder_path = 'D:\\Work_file\\det\\bbox\\whole_groundtruths\\'
pred_file_name_1 = 'D:\\Work_file\\det\\bbox\\atten_unet_48_1205230047_stageoneresult\\'
pred_file_name_2 = 'D:\\Work_file\\det\\bbox\\atten_unet_48_1209145649_stagetwo\\'
confidence = 0.4
for filename in os.listdir(folder_path):
    # print(filename)
    # filename = '02191202220027'
    gt_boxes, shape = get_gtboxes(folder_path, filename)
    pred1_boxes, shape = get_predboxes(pred_file_name_1, filename, confidence)
    pred2_boxes, shape = get_predboxes(pred_file_name_2, filename, confidence)

    set_1, set_2, set_3, set_4, set_5, set_6, set_7 = classify_gt_boxes(gt_boxes, pred1_boxes, pred2_boxes, iou_threshold=0.1)
    volume = create_volume_with_boxes(shape, [set_1, set_2, set_3, set_4, set_5, set_6, set_7], [1, 2, 3, 4, 5, 6, 7])

    spacing = (0.7, 0.7, 0.7)
    affine = np.diag([spacing[0], spacing[1], spacing[2], 1])
    result = tio.ScalarImage(tensor=torch.tensor(volume).unsqueeze(0), affine=(affine))
    result.save(f'D:\\Work_file\\uii_lymph_nodes_data\\all_whole_testing\\{filename[:-4]}-bbox{confidence}.nii.gz')



KeyboardInterrupt: 